In [1]:
import os,sys
from slr.model.st_gcn import st_gcn
import tensorflow as tf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time
from tqdm import tqdm
os.getcwd()

'C:\\Users\\user\\Documents\\GitHub\\slr_proj'

In [2]:
from slr.data.ksl.ksl_adj import Graph

In [4]:
def get_parsed_dataset(tfrec_Dset:tf.data.TFRecordDataset, batch_size = 32):
    image_feature_description = {
        'raw_data': tf.io.FixedLenFeature([], tf.string),
        'data_shape': tf.io.FixedLenFeature([],tf.string),
        'label': tf.io.FixedLenFeature([], tf.string),
    }
    def _parse_image_function(example):
        example = tf.io.parse_single_example(example, image_feature_description)
        example['label'] = tf.cast(example['label'],tf.string)
        example['raw_data'] = tf.reshape(tf.io.decode_raw(example['raw_data'],tf.float32),(-1,137,3))
        return example

    parsed_dataset = tfrec_Dset.map(_parse_image_function).batch(batch_size)
#     parsed_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return parsed_dataset

def get_ksl_dataset_from_tfrec(file, comp=None, batch_size=32):
    dataset = tf.data.TFRecordDataset(file, comp)
    parsed_dataset = get_parsed_dataset(dataset,batch_size)
    parsed_dataset = parsed_dataset.as_numpy_iterator()
    return parsed_dataset

In [7]:
def print_log(str_, print_time=True):
    # print(str_)
    if print_time:
        localtime = time.asctime(time.localtime(time.time()))
        str_ = "[ " + localtime + ' ] ' + str_
    print(str_)
    with open('./log.txt', 'a') as f:
        print(str_, file=f)

In [6]:
dataset = get_ksl_dataset_from_tfrec('./slr/data/gzip_test_with_preprocess.tfrec','GZIP',batch_size=2)

In [3]:
model = st_gcn.Model(channel = 3,
            num_class=100,
            window_size=150,
            num_point=137,
            graph = Graph('spatial')).cuda(0)

In [4]:
# from torchsummary import summary
# summary(model, (3,150,137,1),device='cuda')

In [11]:
from slr.data import DataPath
from slr.data.datagenerator import KeyDataGenerator
from sklearn.model_selection import train_test_split
bat_size=8
class_limit=100
x,y = DataPath(class_limit=class_limit).data


x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=66, test_size=0.3)

train_generator = KeyDataGenerator(x_train,y_train,batch_size = bat_size,scale=True,seq_len=150)
test_generator = KeyDataGenerator(x_test,y_test,batch_size = bat_size, seq_len=150)

In [19]:
train_generator[0][0].reshape((bat_size,-1,137,3))
train_generator[0][1]

array([90, 24, 10, 84, 79,  5, 59, 45, 74, 60, 54, 18, 14, 68, 11, 35, 63,
       99, 46, 27, 23, 64, 84, 32,  8, 30, 62,  0, 36, 11,  8, 88, 75, 87,
       62, 38, 91,  8, 12, 58, 72, 43, 25, 73, 38, 42, 61, 61,  5, 28, 64,
       88, 18, 22,  7, 34,  3, 29, 48, 44, 19, 37, 81, 41,  9, 79, 10, 84,
       57, 69, 26, 84, 99, 68, 33, 60, 48, 20, 76, 44, 10, 96, 52, 34, 13,
       30, 19, 15, 84, 56,  7, 60, 82, 94, 94, 20,  5, 77,  9, 84, 81, 78,
       13, 91, 98,  6, 61, 18, 99, 77, 75, 27, 98, 29, 82, 55, 81, 93, 39,
        6, 71, 15,  3, 16, 46, 29, 74, 40], dtype=int64)

In [ ]:
def st_gcn_train(model, train_generator,test_generator, epochs=10):
    criterion = nn.CrossEntropyLoss().float().cuda(0)
    optimizer = optim.Adam(model.parameters())

    for epoch in tqdm(range(epochs)):
        train_loss = 0.0 
        val_acc = 0.0
        training_accuracy = 0.0
        val_loss = 0.0 
        total = 0

        cnt = 0
        for i,d in tqdm(enumerate(train_generator)):
            model.train()
            cnt += 1
#             x = torch.from_numpy(d['raw_data']).float().unsqueeze(-1).permute(0,3,1,2,4).contiguous().cuda(0)
#             print(x.shape)
#             y = torch.Tensor(list(map(lambda x: int(x.decode()), d['label']))).type(torch.LongTensor).cuda(0)
#             print(y)
            x,y = d
            x = torch.from_numpy(x.reshape((bat_size,-1,137,3,))).float().unsqueeze(-1).permute(0,3,1,2,4).contiguous().cuda(0)
            y = torch.from_numpy(y).cuda(0)
            optimizer.zero_grad()

            # 순전파 + 역전파 + 최적화를 한 후
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            _, outputs = torch.max(outputs, 1)
            training_accuracy += (outputs == y).sum().item() / len(y)
            # print(outputs,y)
            # print(training_accuracy)
            # 통계를 출력합니다.
            train_loss += loss.item()
    
        train_loss = train_loss / len(train_generator)
        training_accuracy = training_accuracy / len(test_generator)
        print_log(f'[{epoch + 1}, {i + 1:5d}] loss: {train_loss:.3f} acc: {training_accuracy:.3f}')
    
    # Validation Loop 
    with torch.no_grad(): 
        model.eval() 
        for i,data in tqdm(enumerate(test_generator)): 
            x, y = data
            x = torch.from_numpy(x.reshape(bat_size,-1,137,3)).float().unsqueeze(-1).permute(0,3,1,2,4).contiguous().cuda(0)
            y = torch.from_numpy(y).cuda(0)


            outputs = model(x) 
            loss = criterion(outputs, y) 

            # The label with the highest value will be our prediction 
            _, outputs = torch.max(outputs, 1) 
            val_loss += loss.item()  
            # total += outputs.size(0) 
            val_acc += (outputs == y).sum().item() / outputs.size(0)


        # Calculate validation loss value 
        val_loss = val_loss/len(test_generator) 

        # Calculate accuracy as the number of correct predictions in the validation batch divided by the total number of predictions done.  
        val_acc = val_acc / len(test_generator)# / total)
        print_log(f"val_loss: {val_loss}, val_accuracy: {val_acc}")

# st_gcn_train(model,dataset,10)
st_gcn_train(model,train_generator,test_generator,200)

  0%|                                                         | 0/200 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  2.60it/s]
2it [00:00,  3.34it/s]
3it [00:00,  3.66it/s]
4it [00:01,  3.83it/s]
5it [00:01,  3.92it/s]
6it [00:01,  3.98it/s]
7it [00:01,  4.02it/s]
8it [00:02,  4.06it/s]
9it [00:02,  4.09it/s]
10it [00:02,  4.10it/s]
11it [00:02,  4.11it/s]
12it [00:03,  4.09it/s]
13it [00:03,  3.94it/s]
14it [00:03,  4.00it/s]
15it [00:03,  4.02it/s]
16it [00:04,  4.06it/s]
17it [00:04,  4.07it/s]
18it [00:04,  4.09it/s]
19it [00:04,  4.10it/s]
20it [00:05,  4.11it/s]
21it [00:05,  4.12it/s]
22it [00:05,  4.13it/s]
23it [00:05,  4.13it/s]
24it [00:05,  4.14it/s]
25it [00:06,  4.14it/s]
26it [00:06,  4.15it/s]
27it [00:06,  4.14it/s]
28it [00:06,  4.15it/s]
29it [00:07,  4.14it/s]
30it [00:07,  4.15it/s]
31it [00:07,  4.13it/s]
32it [00:07,  4.14it/s]
33it [00:08,  4.13it/s]
34it [00:08,  4.14it/s]
35it [00:08,  4.13it/s]
36it [00:08,  4.13it/s]
37it [00:09,  4.13it/s]
38it [00:09,  4

293it [01:12,  4.16it/s]
294it [01:12,  4.17it/s]
295it [01:12,  4.16it/s]
296it [01:13,  4.08it/s]
297it [01:13,  4.08it/s]
298it [01:13,  4.09it/s]
299it [01:13,  4.11it/s]
300it [01:14,  4.11it/s]
301it [01:14,  4.13it/s]
302it [01:14,  4.10it/s]
303it [01:14,  4.01it/s]
304it [01:15,  3.95it/s]
305it [01:15,  4.02it/s]
306it [01:15,  4.07it/s]
307it [01:15,  4.10it/s]
308it [01:16,  4.08it/s]
309it [01:16,  3.97it/s]
310it [01:16,  3.99it/s]
311it [01:16,  3.93it/s]
312it [01:17,  3.88it/s]
313it [01:17,  3.84it/s]
314it [01:17,  3.88it/s]
315it [01:17,  3.89it/s]
316it [01:18,  3.90it/s]
317it [01:18,  3.88it/s]
318it [01:18,  3.91it/s]
319it [01:19,  3.89it/s]
320it [01:19,  3.89it/s]
321it [01:19,  3.86it/s]
322it [01:19,  3.89it/s]
323it [01:20,  3.88it/s]
324it [01:20,  3.90it/s]
325it [01:20,  3.87it/s]
326it [01:20,  3.91it/s]
327it [01:21,  3.98it/s]
328it [01:21,  4.04it/s]
329it [01:21,  4.08it/s]
330it [01:21,  4.12it/s]
331it [01:22,  4.13it/s]
332it [01:22,  4.07it/s]


585it [02:23,  4.16it/s]
586it [02:23,  4.16it/s]
587it [02:23,  4.17it/s]
588it [02:23,  4.15it/s]
589it [02:24,  4.16it/s]
590it [02:24,  4.15it/s]
591it [02:24,  4.16it/s]
592it [02:24,  4.16it/s]
593it [02:25,  4.16it/s]
594it [02:25,  4.16it/s]
595it [02:25,  4.17it/s]
596it [02:25,  4.17it/s]
597it [02:26,  4.17it/s]
598it [02:26,  4.18it/s]
599it [02:26,  4.17it/s]
600it [02:26,  4.17it/s]
601it [02:27,  4.16it/s]
602it [02:27,  4.17it/s]
603it [02:27,  4.16it/s]
604it [02:27,  4.17it/s]
605it [02:27,  4.16it/s]
606it [02:28,  4.17it/s]
607it [02:28,  4.16it/s]
608it [02:28,  4.17it/s]
609it [02:28,  4.15it/s]
610it [02:29,  4.16it/s]
611it [02:29,  4.15it/s]
612it [02:29,  4.16it/s]
613it [02:29,  4.16it/s]
614it [02:30,  4.17it/s]
615it [02:30,  4.16it/s]
616it [02:30,  4.17it/s]
617it [02:30,  4.15it/s]
618it [02:31,  4.16it/s]
619it [02:31,  4.16it/s]
620it [02:31,  4.17it/s]
621it [02:31,  4.17it/s]
622it [02:32,  4.17it/s]
623it [02:32,  4.17it/s]
624it [02:32,  4.17it/s]


[ Mon Aug 22 16:52:38 2022 ] [1,   700] loss: 3.859 acc: 0.198



0it [00:00, ?it/s]
1it [00:00,  4.13it/s]
2it [00:00,  4.15it/s]
3it [00:00,  4.15it/s]
4it [00:00,  4.16it/s]
5it [00:01,  4.15it/s]
6it [00:01,  4.10it/s]
7it [00:01,  4.10it/s]
8it [00:01,  4.12it/s]
9it [00:02,  4.13it/s]
10it [00:02,  4.15it/s]
11it [00:02,  4.14it/s]
12it [00:02,  4.15it/s]
13it [00:03,  4.15it/s]
14it [00:03,  4.16it/s]
15it [00:03,  4.15it/s]
16it [00:03,  4.16it/s]
17it [00:04,  4.15it/s]
18it [00:04,  4.17it/s]
19it [00:04,  4.16it/s]
20it [00:04,  4.16it/s]
21it [00:05,  4.15it/s]
22it [00:05,  4.16it/s]
23it [00:05,  4.15it/s]
24it [00:05,  4.17it/s]
25it [00:06,  4.16it/s]
26it [00:06,  4.16it/s]
27it [00:06,  4.17it/s]
28it [00:06,  4.17it/s]
29it [00:06,  4.17it/s]
30it [00:07,  4.17it/s]
31it [00:07,  4.16it/s]
32it [00:07,  4.15it/s]
33it [00:07,  4.16it/s]
34it [00:08,  4.15it/s]
35it [00:08,  4.16it/s]
36it [00:08,  4.16it/s]
37it [00:08,  4.17it/s]
38it [00:09,  4.16it/s]
39it [00:09,  4.17it/s]
40it [00:09,  4.16it/s]
41it [00:09,  4.16it/s]
42it 

296it [01:11,  4.16it/s]
297it [01:11,  4.16it/s]
298it [01:11,  4.16it/s]
299it [01:11,  4.16it/s]
300it [01:12,  4.15it/s]
301it [01:12,  4.16it/s]
302it [01:12,  4.16it/s]
303it [01:12,  4.16it/s]
304it [01:13,  4.16it/s]
305it [01:13,  4.16it/s]
306it [01:13,  4.16it/s]
307it [01:13,  4.16it/s]
308it [01:14,  4.16it/s]
309it [01:14,  4.16it/s]
310it [01:14,  4.15it/s]
311it [01:14,  4.16it/s]
312it [01:15,  4.16it/s]
313it [01:15,  4.16it/s]
314it [01:15,  4.14it/s]
315it [01:15,  4.15it/s]
316it [01:16,  4.16it/s]
317it [01:16,  4.15it/s]
318it [01:16,  4.16it/s]
319it [01:16,  4.15it/s]
320it [01:16,  4.16it/s]
321it [01:17,  4.15it/s]
322it [01:17,  4.15it/s]
323it [01:17,  4.15it/s]
324it [01:17,  4.16it/s]
325it [01:18,  4.15it/s]
326it [01:18,  4.16it/s]
327it [01:18,  4.16it/s]
328it [01:18,  4.17it/s]
329it [01:19,  4.16it/s]
330it [01:19,  4.16it/s]
331it [01:19,  4.15it/s]
332it [01:19,  4.16it/s]
333it [01:20,  4.15it/s]
334it [01:20,  4.16it/s]
335it [01:20,  4.16it/s]


588it [02:21,  4.16it/s]
589it [02:21,  4.15it/s]
590it [02:21,  4.16it/s]
591it [02:22,  4.16it/s]
592it [02:22,  4.16it/s]
593it [02:22,  4.15it/s]
594it [02:22,  4.16it/s]
595it [02:23,  4.15it/s]
596it [02:23,  4.16it/s]
597it [02:23,  4.15it/s]
598it [02:23,  4.16it/s]
599it [02:24,  4.15it/s]
600it [02:24,  4.16it/s]
601it [02:24,  4.17it/s]
602it [02:24,  4.16it/s]
603it [02:24,  4.16it/s]
604it [02:25,  4.16it/s]
605it [02:25,  4.16it/s]
606it [02:25,  4.16it/s]
607it [02:25,  4.17it/s]
608it [02:26,  4.16it/s]
609it [02:26,  4.17it/s]
610it [02:26,  4.16it/s]
611it [02:26,  4.16it/s]
612it [02:27,  4.15it/s]
613it [02:27,  4.16it/s]
614it [02:27,  4.16it/s]
615it [02:27,  4.16it/s]
616it [02:28,  4.16it/s]
617it [02:28,  4.16it/s]
618it [02:28,  4.16it/s]
619it [02:28,  4.17it/s]
620it [02:29,  4.16it/s]
621it [02:29,  4.17it/s]
622it [02:29,  4.16it/s]
623it [02:29,  4.16it/s]
624it [02:30,  4.16it/s]
625it [02:30,  4.16it/s]
626it [02:30,  4.16it/s]
627it [02:30,  4.17it/s]


[ Mon Aug 22 16:55:27 2022 ] [2,   700] loss: 3.749 acc: 0.231



0it [00:00, ?it/s]
1it [00:00,  4.18it/s]
2it [00:00,  4.16it/s]
3it [00:00,  4.15it/s]
4it [00:00,  4.09it/s]
5it [00:01,  3.90it/s]
6it [00:01,  3.88it/s]
7it [00:01,  3.93it/s]
8it [00:01,  4.00it/s]
9it [00:02,  4.05it/s]
10it [00:02,  4.07it/s]
11it [00:02,  4.10it/s]
12it [00:02,  4.11it/s]
13it [00:03,  4.12it/s]
14it [00:03,  4.10it/s]
15it [00:03,  4.10it/s]
16it [00:03,  4.12it/s]
17it [00:04,  4.04it/s]
18it [00:04,  4.08it/s]
19it [00:04,  4.08it/s]
20it [00:04,  4.10it/s]
21it [00:05,  4.10it/s]
22it [00:05,  4.10it/s]
23it [00:05,  4.10it/s]
24it [00:05,  4.03it/s]
25it [00:06,  3.96it/s]
26it [00:06,  4.00it/s]
27it [00:06,  4.02it/s]
28it [00:06,  4.02it/s]
29it [00:07,  4.05it/s]
30it [00:07,  4.01it/s]
31it [00:07,  3.96it/s]
32it [00:07,  3.93it/s]
33it [00:08,  3.94it/s]
34it [00:08,  3.99it/s]
35it [00:08,  3.93it/s]
36it [00:08,  3.84it/s]
37it [00:09,  3.79it/s]
38it [00:09,  3.76it/s]
39it [00:09,  3.73it/s]
40it [00:10,  3.72it/s]
41it [00:10,  3.72it/s]
42it 

296it [01:12,  4.31it/s]
297it [01:12,  4.31it/s]
298it [01:13,  4.32it/s]
299it [01:13,  4.33it/s]
300it [01:13,  4.28it/s]
301it [01:13,  4.29it/s]
302it [01:14,  4.31it/s]
303it [01:14,  4.31it/s]
304it [01:14,  4.32it/s]
305it [01:14,  4.32it/s]
306it [01:14,  4.33it/s]
307it [01:15,  4.33it/s]
308it [01:15,  4.33it/s]
309it [01:15,  4.33it/s]
310it [01:15,  4.33it/s]
311it [01:16,  4.28it/s]
312it [01:16,  4.30it/s]
313it [01:16,  4.15it/s]
314it [01:16,  4.01it/s]
315it [01:17,  3.92it/s]
316it [01:17,  3.83it/s]
317it [01:17,  3.91it/s]
318it [01:17,  4.01it/s]
319it [01:18,  4.09it/s]
320it [01:18,  4.15it/s]
321it [01:18,  4.08it/s]
322it [01:18,  3.97it/s]
323it [01:19,  3.92it/s]
324it [01:19,  3.89it/s]
325it [01:19,  3.99it/s]
326it [01:19,  4.05it/s]
327it [01:20,  4.11it/s]
328it [01:20,  4.16it/s]
329it [01:20,  4.14it/s]
330it [01:20,  4.14it/s]
331it [01:21,  4.18it/s]
332it [01:21,  4.05it/s]
333it [01:21,  4.05it/s]
334it [01:21,  4.00it/s]
335it [01:22,  4.09it/s]
